In [9]:
from src.dataio import load_data
from pprint import pprint
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from keras.preprocessing.sequence import pad_sequences

2024-01-06 21:43:08.554622: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 21:43:08.789199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 21:43:10.209276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
MAX_LEN = 256

In [4]:
trn, dev, tst = load_data('data/fn1.7_parsed/')

# of instances in trn: 19391
# of instances in dev: 2272
# of instances in tst: 6714
data example: [['Does', 'Iran', '<tgt>', 'have', '</tgt>', 'the', 'infrastructure', 'necessary', 'to', 'produce', 'nuclear', 'weapons', '?'], ['_', '_', '_', 'have.v', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['_', '_', '_', 'Possession', '_', '_', '_', '_', '_', '_', '_', '_', '_'], ['O', 'B-Owner', 'X', 'O', 'X', 'B-Possession', 'I-Possession', 'I-Possession', 'I-Possession', 'I-Possession', 'I-Possession', 'I-Possession', 'O']]


In [ ]:
def bert_tokenizer(text):
    orig_tokens = text.split(' ')
    bert_tokens = []
    orig_to_tok_map = []
    bert_tokens.append("[CLS]")
    for orig_token in orig_tokens:
        orig_to_tok_map.append(len(bert_tokens))
        bert_tokens.extend(self.tokenizer.tokenize(orig_token))
    bert_tokens.append("[SEP]")

    return orig_tokens, bert_tokens, orig_to_tok_map


In [7]:
def convert_to_bert_input_JointShallowSemanticParsing( input_data):
    tokenized_texts, lus, senses, args = [],[],[],[]
    orig_tok_to_maps = []
    for i in range(len(input_data)):    
        data = input_data[i]
        text = ' '.join(data[0])
        orig_tokens, bert_tokens, orig_to_tok_map = self.bert_tokenizer(text)

        orig_tok_to_maps.append(orig_to_tok_map)
        tokenized_texts.append(bert_tokens)

        ori_lus = data[1]    
        lu_sequence = []
        for i in range(len(bert_tokens)):
            if i in orig_to_tok_map:
                idx = orig_to_tok_map.index(i)
                l = ori_lus[idx]
                lu_sequence.append(l)
            else:
                lu_sequence.append('_')
        lus.append(lu_sequence)        

        if self.mode == 'train':
            ori_senses, ori_args = data[2], data[3]
            sense_sequence, arg_sequence = [],[]
            for i in range(len(bert_tokens)):
                if i in orig_to_tok_map:
                    idx = orig_to_tok_map.index(i)
                    fr = ori_senses[idx]
                    sense_sequence.append(fr)
                    ar = ori_args[idx]
                    arg_sequence.append(ar)
                else:
                    sense_sequence.append('_')
                    arg_sequence.append('X')
            senses.append(sense_sequence)
            args.append(arg_sequence)

    input_ids = pad_sequences([self.tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                            maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    
    orig_tok_to_maps = pad_sequences(orig_tok_to_maps, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post", value=-1)
    
    if self.mode =='train':
        if self.srl == 'propbank-dp':
            arg_ids = pad_sequences([[self.arg2idx.get(ar) for ar in arg] for arg in args],
                                    maxlen=MAX_LEN, value=self.arg2idx["X"], padding="post",
                                    dtype="long", truncating="post")
        elif self.srl == 'framenet-argid':
            arg_ids = pad_sequences([[self.bio_argument2idx.get(ar) for ar in arg] for arg in args],
                                    maxlen=MAX_LEN, value=self.bio_argument2idx["X"], padding="post",
                                    dtype="long", truncating="post")
        else:
            arg_ids = pad_sequences([[self.bio_arg2idx.get(ar) for ar in arg] for arg in args],
                                    maxlen=MAX_LEN, value=self.bio_arg2idx["X"], padding="post",
                                    dtype="long", truncating="post")

    lu_seq, sense_seq = [],[]
    for sent_idx in range(len(lus)):
        lu_items = lus[sent_idx]
        lu = []
        for idx in range(len(lu_items)):
            if lu_items[idx] != '_':
                if len(lu) == 0:
                    if self.mode != 'train' and self.masking == False:
                        lu.append(1)
                    else:
                        lu.append(self.lu2idx[lu_items[idx]])
                        
        lu_seq.append(lu)
        
        if self.mode == 'train':
            sense_items, arg_items = senses[sent_idx], args[sent_idx]
            sense = []
            for idx in range(len(sense_items)):
                if sense_items[idx] != '_':
                    if len(sense) == 0:
                        sense.append(self.sense2idx[sense_items[idx]])
            sense_seq.append(sense)

    attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
    token_type_ids = [[0 if idx > 0 else 1 for idx in input_id]for input_id in input_ids]
    
    data_inputs = torch.tensor(input_ids)
    data_orig_tok_to_maps = torch.tensor(orig_tok_to_maps)
    data_lus = torch.tensor(lu_seq)
    data_token_type_ids = torch.tensor(token_type_ids)
    data_masks = torch.tensor(attention_masks)
    
    if self.mode == 'train':
        data_senses = torch.tensor(sense_seq)
        data_args = torch.tensor(arg_ids)
        bert_inputs = TensorDataset(data_inputs, data_orig_tok_to_maps, data_lus, data_senses, data_args, data_token_type_ids, data_masks)
    else:
        bert_inputs = TensorDataset(data_inputs, data_orig_tok_to_maps, data_lus, data_token_type_ids, data_masks)
    return bert_inputs

In [11]:
convert_to_bert_input_JointShallowSemanticParsing(trn)

TypeError: convert_to_bert_input_JointShallowSemanticParsing() missing 1 required positional argument: 'input_data'